In [1]:
import pandas as pd
import numpy as np
import math
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"

MoM FFBL Weekly Rankings Replacement 

In [6]:
"""
MoM FFBL Weekly Rankings Replacement 2pt system
"""
matchups_query = """SELECT * FROM dev.regseasonmatchups"""
weekly_points_query = """SELECT * FROM dev.weeklyteampoints"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams"""
settings_query = """SELECT * FROM dev.leaguesettings"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
weekly_points = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    weekly_points_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
settings = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)

matchups_a = matchups.copy()
matchups_b = matchups.copy()

matchups_b_cols = list(matchups_b.columns)

rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"

matchups_b.rename(columns=rename_columns, inplace=True)

matchups = pd.concat([matchups_a, matchups_b])

matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)

matchups.reset_index(drop=True, inplace=True)

matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
        "team_a_grade",
        "team_b_grade",
    ],
    axis=1,
    inplace=True,
)

weekly_points["team_key"] = (
    weekly_points["game_id"].astype(str)
    + ".l."
    + weekly_points["league_id"].astype(str)
    + ".t."
    + weekly_points["team_id"].astype(str)
)

matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)

matchups["2pt_system_weekly_rank"] = matchups.groupby(["week", "game_id"])[
    "team_a_points"
].rank("first", ascending=False)

matchups["2pt_score"] = np.where(matchups["2pt_system_weekly_rank"] <= 5, 1, 0)

matchups = matchups.merge(
    weekly_points,
    how="outer",
    left_on=["team_a_team_key", "week"],
    right_on=["team_key", "week"],
    suffixes=("", "_wkly_pts"),
)

matchups["game_id"] = matchups["game_id"].fillna(matchups["game_id_wkly_pts"])
matchups["team_a_team_key"] = matchups["team_a_team_key"].fillna(matchups["team_key"])
matchups["team_a_points"] = matchups["team_a_points"].fillna(matchups["final_points"])
matchups["team_a_projected_points"] = matchups["team_a_projected_points"].fillna(
    matchups["projected_points"]
)

matchups = matchups.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)

matchups["team_a_name"] = matchups["name"].fillna(matchups["team_a_team_key"])
matchups["team_a_nickname"] = matchups["nickname"].fillna(matchups["team_a_team_key"])

matchups["game_id_a"] = matchups["game_id_x"].fillna(matchups["game_id_y"])

matchups.drop(["name", "nickname"], axis=1, inplace=True)

matchups = matchups.merge(
    teams, how="outer", left_on="team_b_team_key", right_on="team_key"
)

matchups["team_b_name"] = matchups["name"].fillna(matchups["team_b_team_key"])
matchups["team_b_nickname"] = matchups["nickname"].fillna(matchups["team_b_team_key"])

matchups.drop(["name", "nickname"], axis=1, inplace=True)

matchups["game_id"] = matchups["game_id_a"]

matchups = matchups[
    [
        "game_id",
        "team_a_team_key",
        "team_a_name",
        "team_a_nickname",
        "week",
        "team_a_points",
        "team_a_projected_points",
        "team_b_team_key",
        "team_b_name",
        "team_b_nickname",
        "team_b_points",
        "team_b_projected_points",
        "win_loss",
        "2pt_score",
    ]
]

weeks = list(matchups["week"].unique())
weeks.sort()

regular_season = matchups[matchups["week"] == 1]

for wk in weeks[1:]:
    reg_season_week = matchups[
        [
            "team_a_team_key",
            "week",
            "team_a_points",
            "team_a_projected_points",
            "team_b_team_key",
            "team_b_name",
            "team_b_nickname",
            "team_b_points",
            "team_b_projected_points",
            "win_loss",
            "2pt_score",
        ]
    ][matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )

reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_score" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)


regular_season.insert(3, "team_a_pts_ttl", regular_season[team_pts].sum(axis=1))
regular_season.insert(4, "team_a_pro_pts_ttl", regular_season[team_pro_pts].sum(axis=1))
regular_season.insert(
    5, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    6, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(3, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(9, "week_1", week_1)
regular_season.insert(
    3,
    "w_l_rank",
    regular_season.groupby(["game_id"])["wins"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    3,
    "2pt_rank",
    regular_season.groupby(["game_id"])["2pt_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    6,
    "pts_rank",
    regular_season.groupby(["game_id"])["team_a_pts_ttl"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    8,
    "pro_pts_rank",
    regular_season.groupby(["game_id"])["team_a_pro_pts_ttl"]
    .rank(method="min", ascending=False)
    .astype(int),
)

df_2pt = regular_season[regular_season["game_id"] >= 390]
df_2pt.insert(0, "tuple", df_2pt[["2pt_rank", "pts_rank"]].apply(tuple, axis=1))

df_h2h = regular_season[regular_season["game_id"] < 390]
df_h2h.insert(0, "tuple", df_h2h[["w_l_rank", "pts_rank"]].apply(tuple, axis=1))

regular_season = pd.concat([df_2pt, df_h2h])

regular_season.insert(
    0,
    "reg_season_rank",
    regular_season.groupby(["game_id"])["tuple"]
    .rank(method="min", ascending=True)
    .astype(int),
)

regular_season.sort_values(
    ["reg_season_rank", "2pt_total", "team_a_pts_ttl"],
    ascending=[True, False, False],
    inplace=True,
)

rename_columns_2 = {}
for col in regular_season.columns:
    if "team_a_" in col:
        rename_columns_2[col] = col[len("team_a_") :]
    elif "team_b_" in col:
        rename_columns_2[col] = f'opp_{col[len("team_a_"):]}'

regular_season.rename(columns=rename_columns_2, inplace=True)

rename_columns_3 = {}
for col in regular_season.columns[15:]:
    if not str(col)[-1].isnumeric():
        rename_columns_3[col] = f"{col}_1"

regular_season.rename(columns=rename_columns_3, inplace=True)

rename_columns_4 = {"name": "team", "nickname": "mgr"}
for col in regular_season.columns[15:]:
    if "opp_name" in col:
        rename_columns_4[col] = f"opp_team{col[8:]}"
    elif "opp_nickname" in col:
        rename_columns_4[col] = f"opp_mgr{col[12:]}"
    elif "opp_team_key_" in col:
        rename_columns_4[col] = f"opp_key{col[12:]}"
    elif "_projected_points" in col:
        rename_columns_4[col] = f"opp_pro_pts{col[len('opp_projected_points'):]}"
    elif "projected_points_" in col:
        rename_columns_4[col] = f"pro_pts{col[len('projected_points'):]}"
    elif "_points" in col:
        rename_columns_4[col] = f"opp_pts{col[len('opp_points'):]}"
    elif "points_" in col:
        rename_columns_4[col] = f"pts{col[len('points'):]}"

regular_season.rename(columns=rename_columns_4, inplace=True)

regular_season.drop("tuple", axis=1, inplace=True)

regular_season = regular_season[
    [
        "game_id",
        "reg_season_rank",
        "team_key",
        "team",
        "mgr",
        "2pt_rank",
        "2pt_total",
        "w_l_rank",
        "wins",
        "losses",
        "pts_rank",
        "pts_ttl",
        "pro_pts_rank",
        "pro_pts_ttl",
        "week_1",
        "pts_1",
        "pro_pts_1",
        "opp_key_1",
        "opp_team_1",
        "opp_mgr_1",
        "opp_pts_1",
        "opp_pro_pts_1",
        "win_loss_1",
        "2pt_score_1",
        "week_2",
        "pts_2",
        "pro_pts_2",
        "opp_key_2",
        "opp_team_2",
        "opp_mgr_2",
        "opp_pts_2",
        "opp_pro_pts_2",
        "win_loss_2",
        "2pt_score_2",
        "week_3",
        "pts_3",
        "pro_pts_3",
        "opp_key_3",
        "opp_team_3",
        "opp_mgr_3",
        "opp_pts_3",
        "opp_pro_pts_3",
        "win_loss_3",
        "2pt_score_3",
        "week_4",
        "pts_4",
        "pro_pts_4",
        "opp_key_4",
        "opp_team_4",
        "opp_mgr_4",
        "opp_pts_4",
        "opp_pro_pts_4",
        "win_loss_4",
        "2pt_score_4",
        "week_5",
        "pts_5",
        "pro_pts_5",
        "opp_key_5",
        "opp_team_5",
        "opp_mgr_5",
        "opp_pts_5",
        "opp_pro_pts_5",
        "win_loss_5",
        "2pt_score_5",
        "week_6",
        "pts_6",
        "pro_pts_6",
        "opp_key_6",
        "opp_team_6",
        "opp_mgr_6",
        "opp_pts_6",
        "opp_pro_pts_6",
        "win_loss_6",
        "2pt_score_6",
        "week_7",
        "pts_7",
        "pro_pts_7",
        "opp_key_7",
        "opp_team_7",
        "opp_mgr_7",
        "opp_pts_7",
        "opp_pro_pts_7",
        "win_loss_7",
        "2pt_score_7",
        "week_8",
        "pts_8",
        "pro_pts_8",
        "opp_key_8",
        "opp_team_8",
        "opp_mgr_8",
        "opp_pts_8",
        "opp_pro_pts_8",
        "win_loss_8",
        "2pt_score_8",
        "week_9",
        "pts_9",
        "pro_pts_9",
        "opp_key_9",
        "opp_team_9",
        "opp_mgr_9",
        "opp_pts_9",
        "opp_pro_pts_9",
        "win_loss_9",
        "2pt_score_9",
        "week_10",
        "pts_10",
        "pro_pts_10",
        "opp_key_10",
        "opp_team_10",
        "opp_mgr_10",
        "opp_pts_10",
        "opp_pro_pts_10",
        "win_loss_10",
        "2pt_score_10",
        "week_11",
        "pts_11",
        "pro_pts_11",
        "opp_key_11",
        "opp_team_11",
        "opp_mgr_11",
        "opp_pts_11",
        "opp_pro_pts_11",
        "win_loss_11",
        "2pt_score_11",
        "week_12",
        "pts_12",
        "pro_pts_12",
        "opp_key_12",
        "opp_team_12",
        "opp_mgr_12",
        "opp_pts_12",
        "opp_pro_pts_12",
        "win_loss_12",
        "2pt_score_12",
        "week_13",
        "pts_13",
        "pro_pts_13",
        "opp_key_13",
        "opp_team_13",
        "opp_mgr_13",
        "opp_pts_13",
        "opp_pro_pts_13",
        "win_loss_13",
        "2pt_score_13",
        "week_14",
        "pts_14",
        "pro_pts_14",
        "opp_key_14",
        "opp_team_14",
        "opp_mgr_14",
        "opp_pts_14",
        "opp_pro_pts_14",
        "win_loss_14",
        "2pt_score_14",
        "week_15",
        "pts_15",
        "pro_pts_15",
        "opp_key_15",
        "opp_team_15",
        "opp_mgr_15",
        "opp_pts_15",
        "opp_pro_pts_15",
        "win_loss_15",
        "2pt_score_15",
        "week_16",
        "pts_16",
        "pro_pts_16",
        "opp_key_16",
        "opp_team_16",
        "opp_mgr_16",
        "opp_pts_16",
        "opp_pro_pts_16",
        "win_loss_16",
        "2pt_score_16",
        "week_17",
        "pts_17",
        "pro_pts_17",
        "opp_key_17",
        "opp_team_17",
        "opp_mgr_17",
        "opp_pts_17",
        "opp_pro_pts_17",
        "win_loss_17",
        "2pt_score_17",
    ]
]

game_keys = list(regular_season["game_id"].unique())

for key in game_keys:
    one_season = regular_season[regular_season["game_id"] == key]
    DatabaseCursor(PATH, options=OPTION_PROD).copy_table_to_postgres_new(
        one_season, f"matchup_board_{int(key)}", first_time="yes"
    )

Successfully pulled: SELECT * FROM dev.regseasonmatchups
Successfully pulled: SELECT * FROM dev.weeklyteampoints
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams


/tmp/ipykernel_4233/1624265684.py:221: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  regular_season.insert(


Upload successful: matchup_board_348
Upload successful: matchup_board_406
Upload successful: matchup_board_371
Upload successful: matchup_board_331
Upload successful: matchup_board_380
Upload successful: matchup_board_399
Upload successful: matchup_board_390
Upload successful: matchup_board_273
Upload successful: matchup_board_314
Upload successful: matchup_board_359


In [3]:
game_id_406 = regular_season.groupby("game_id").get_group(399)
game_id_406

,game_id,reg_season_rank,team_key,team,mgr,2pt_rank,2pt_total,w_l_rank,wins,losses,pts_rank,pts_ttl,pro_pts_rank,pro_pts_ttl,week_1,pts_1,pro_pts_1,opp_key_1,opp_team_1,opp_mgr_1,opp_pts_1,opp_pro_pts_1,win_loss_1,2pt_score_1,week_2,pts_2,pro_pts_2,opp_key_2,opp_team_2,opp_mgr_2,opp_pts_2,opp_pro_pts_2,win_loss_2,2pt_score_2,week_3,pts_3,pro_pts_3,opp_key_3,opp_team_3,opp_mgr_3,opp_pts_3,opp_pro_pts_3,win_loss_3,2pt_score_3,week_4,pts_4,pro_pts_4,opp_key_4,opp_team_4,opp_mgr_4,opp_pts_4,opp_pro_pts_4,win_loss_4,2pt_score_4,week_5,pts_5,pro_pts_5,opp_key_5,opp_team_5,opp_mgr_5,opp_pts_5,opp_pro_pts_5,win_loss_5,2pt_score_5,week_6,pts_6,pro_pts_6,opp_key_6,opp_team_6,opp_mgr_6,opp_pts_6,opp_pro_pts_6,win_loss_6,2pt_score_6,week_7,pts_7,pro_pts_7,opp_key_7,opp_team_7,opp_mgr_7,opp_pts_7,opp_pro_pts_7,win_loss_7,2pt_score_7,week_8,pts_8,pro_pts_8,opp_key_8,opp_team_8,opp_mgr_8,opp_pts_8,opp_pro_pts_8,win_loss_8,2pt_score_8,week_9,pts_9,pro_pts_9,opp_key_9,opp_team_9,opp_mgr_9,opp_pts_9,opp_pro_pts_9,win_loss_9,2pt_score_9,week_10,pts_10,pro_pts_10,opp_key_10,opp_team_10,opp_mgr_10,opp_pts_10,opp_pro_pts_10,win_loss_10,2pt_score_10,week_11,pts_11,pro_pts_11,opp_key_11,opp_team_11,opp_mgr_11,opp_pts_11,opp_pro_pts_11,win_loss_11,2pt_score_11,week_12,pts_12,pro_pts_12,opp_key_12,opp_team_12,opp_mgr_12,opp_pts_12,opp_pro_pts_12,win_loss_12,2pt_score_12,week_13,pts_13,pro_pts_13,opp_key_13,opp_team_13,opp_mgr_13,opp_pts_13,opp_pro_pts_13,win_loss_13,2pt_score_13,week_14,pts_14,pro_pts_14,opp_key_14,opp_team_14,opp_mgr_14,opp_pts_14,opp_pro_pts_14,win_loss_14,2pt_score_14,week_15,pts_15,pro_pts_15,opp_key_15,opp_team_15,opp_mgr_15,opp_pts_15,opp_pro_pts_15,win_loss_15,2pt_score_15,week_16,pts_16,pro_pts_16,opp_key_16,opp_team_16,opp_mgr_16,opp_pts_16,opp_pro_pts_16,win_loss_16,2pt_score_16,week_17,pts_17,pro_pts_17,opp_key_17,opp_team_17,opp_mgr_17,opp_pts_17,opp_pro_pts_17,win_loss_17,2pt_score_17
87,399.0,1,399.l.12514.t.7,Push Ups in Prison,Pat,1,20.0,2,10,3,2,1997.48,2,1945.74,1,111.54,126.07,399.l.12514.t.5,Krispy Kareem 🍩,Greg,117.48,116.11,L,0.0,2,146.18,120.34,399.l.12514.t.6,Poor Decisions 😢,Pete,197.44,127.52,L,1.0,3,169.30,129.33,399.l.12514.t.10,Gronk's Gurley Chubb,Jeremy,130.50,119.53,W,1.0,4,125.54,129.50,399.l.12514.t.8,Table scraps,James,97.82,134.51,W,1.0,5,151.40,127.05,399.l.12514.t.9,Jonesin' 4 A Win,Chris,109.54,109.02,W,1.0,6,123.20,113.59,399.l.12514.t.1,Fantasy Sidelines,Tim,87.18,120.21,W,1.0,7,122.70,123.70,399.l.12514.t.2,FreshPrinceOfHelaire,Ryan,90.82,108.10,W,1.0,8,140.04,119.64,399.l.12514.t.3,Butker is my lube,Kevin,75.92,108.67,W,1.0,9,98.38,119.36,399.l.12514.t.4,Beats by DeAndre,Wes,84.64,119.07,W,0.0,10,85.38,117.24,399.l.12514.t.5,Krispy Kareem 🍩,Greg,139.06,122.07,L,0.0,11,115.02,119.69,399.l.12514.t.6,Poor Decisions 😢,Pete,103.02,125.40,W,1.0,12,139.08,120.53,399.l.12514.t.10,Gronk's Gurley Chubb,Jeremy,124.80,114.54,W,1.0,13,118.02,118.04,399.l.12514.t.8,Table scraps,James,94.06,108.54,W,1.0,14,89.52,106.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,143.66,126.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,118.52,128.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,399.0,2,399.l.12514.t.5,Krispy Kareem 🍩,Greg,1,20.0,1,11,2,3,1890.86,3,1915.47,1,117.48,116.11,399.l.12514.t.7,Push Ups in Prison,Pat,111.54,126.07,W,1.0,2,121.80,117.01,399.l.12514.t.8,Table scraps,James,119.76,125.66,W,1.0,3,125.44,116.52,399.l.12514.t.9,Jonesin' 4 A Win,Chris,120.78,116.92,W,1.0,4,97.92,118.86,399.l.12514.t.1,Fantasy Sidelines,Tim,88.20,108.36,W,0.0,5,95.52,116.19,399.l.12514.t.2,FreshPrinceOfHelaire,Ryan,74.66,111.97,W,0.0,6,96.08,117.72,399.l.12514.t.3,Butker is my lube,Kevin,91.42,118.67,W,0.0,7,149.58,124.52,399.l.12514.t.4,Beats by DeAndre,Wes,99.76,120.61,W,1.0,8,101.26,124.07,399.l.12514.t.10,Gronk's Gurley Chubb,Jeremy,111.14,101.72,L,0.0,9,116.40,122.01,399.l.12514.t.6,Poor Decisions 😢,Pete,171.72,119.96,L,1.0,10,139.06,122.07,399.l.12514.t.7,Push Ups in Prison,Pat,85.38,117.24,W,1.0,11,121.28,117.46,39

In [7]:
season_273 = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres('SELECT * FROM prod."matchup_board_273"')
season_273

Successfully pulled: SELECT * FROM prod."matchup_board_273"


,game_id,reg_season_rank,team_key,team,mgr,2pt_rank,2pt_total,w_l_rank,wins,losses,pts_rank,pts_ttl,pro_pts_rank,pro_pts_ttl,week_1,pts_1,pro_pts_1,opp_key_1,opp_team_1,opp_mgr_1,opp_pts_1,opp_pro_pts_1,win_loss_1,2pt_score_1,week_2,pts_2,pro_pts_2,opp_key_2,opp_team_2,opp_mgr_2,opp_pts_2,opp_pro_pts_2,win_loss_2,2pt_score_2,week_3,pts_3,pro_pts_3,opp_key_3,opp_team_3,opp_mgr_3,opp_pts_3,opp_pro_pts_3,win_loss_3,2pt_score_3,week_4,pts_4,pro_pts_4,opp_key_4,opp_team_4,opp_mgr_4,opp_pts_4,opp_pro_pts_4,win_loss_4,2pt_score_4,week_5,pts_5,pro_pts_5,opp_key_5,opp_team_5,opp_mgr_5,opp_pts_5,opp_pro_pts_5,win_loss_5,2pt_score_5,week_6,pts_6,pro_pts_6,opp_key_6,opp_team_6,opp_mgr_6,opp_pts_6,opp_pro_pts_6,win_loss_6,2pt_score_6,week_7,pts_7,pro_pts_7,opp_key_7,opp_team_7,opp_mgr_7,opp_pts_7,opp_pro_pts_7,win_loss_7,2pt_score_7,week_8,pts_8,pro_pts_8,opp_key_8,opp_team_8,opp_mgr_8,opp_pts_8,opp_pro_pts_8,win_loss_8,2pt_score_8,week_9,pts_9,pro_pts_9,opp_key_9,opp_team_9,opp_mgr_9,opp_pts_9,opp_pro_pts_9,win_loss_9,2pt_score_9,week_10,pts_10,pro_pts_10,opp_key_10,opp_team_10,opp_mgr_10,opp_pts_10,opp_pro_pts_10,win_loss_10,2pt_score_10,week_11,pts_11,pro_pts_11,opp_key_11,opp_team_11,opp_mgr_11,opp_pts_11,opp_pro_pts_11,win_loss_11,2pt_score_11,week_12,pts_12,pro_pts_12,opp_key_12,opp_team_12,opp_mgr_12,opp_pts_12,opp_pro_pts_12,win_loss_12,2pt_score_12,week_13,pts_13,pro_pts_13,opp_key_13,opp_team_13,opp_mgr_13,opp_pts_13,opp_pro_pts_13,win_loss_13,2pt_score_13,week_14,pts_14,pro_pts_14,opp_key_14,opp_team_14,opp_mgr_14,opp_pts_14,opp_pro_pts_14,win_loss_14,2pt_score_14,week_15,pts_15,pro_pts_15,opp_key_15,opp_team_15,opp_mgr_15,opp_pts_15,opp_pro_pts_15,win_loss_15,2pt_score_15,week_16,pts_16,pro_pts_16,opp_key_16,opp_team_16,opp_mgr_16,opp_pts_16,opp_pro_pts_16,win_loss_16,2pt_score_16,week_17,pts_17,pro_pts_17,opp_key_17,opp_team_17,opp_mgr_17,opp_pts_17,opp_pro_pts_17,win_loss_17,2pt_score_17
0,273,1,273.l.777818.t.7,Goons,Jake,1,19,1,11,3,2,1619.92,3,1553.41,1,105.06,83.85,273.l.777818.t.12,273.l.777818.t.12,273.l.777818.t.12,82.64,93.65,W,1,2,96.46,96.64,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,90.50,96.70,W,0,3,78.80,98.21,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,71.92,112.04,W,0,4,114.56,88.40,273.l.777818.t.10,273.l.777818.t.10,273.l.777818.t.10,101.04,104.59,W,1,5,93.60,100.68,273.l.777818.t.11,La Flama Blanca,Ryan,60.76,102.69,W,0,6,101.16,91.94,273.l.777818.t.1,Taco,James,111.28,74.56,L,0,7,114.88,93.80,273.l.777818.t.2,273.l.777818.t.2,273.l.777818.t.2,88.92,102.46,W,1,8,94.78,97.84,273.l.777818.t.3,Cudde2,Tim,83.50,83.63,W,1,9,94.58,92.55,273.l.777818.t.4,The Fear Boners,Wes,97.66,85.75,L,0,10,111.04,111.20,273.l.777818.t.5,Role players for day,Pat,105.84,85.23,W,1,11,104.64,90.66,273.l.777818.t.6,ELE,Pete,78.82,72.39,W,1,12,79.82,98.40,273.l.777818.t.12,273.l.777818.t.12,273.l.777818.t.12,80.92,86.40,L,0,13,102.30,107.34,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,78.80,90.23,W,1,14,128.08,98.93,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,75.84,99.01,W,1,15.0,88.50,106.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,111.66,96.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,273,2,273.l.777818.t.10,273.l.777818.t.10,273.l.777818.t.10,1,19,2,9,5,1,1671.48,2,1630.67,1,106.08,101.56,273.l.777818.t.4,The Fear Boners,Wes,85.36,92.69,W,1,2,131.84,99.99,273.l.777818.t.5,Role players for day,Pat,56.26,100.65,W,1,3,105.58,94.60,273.l.777818.t.6,ELE,Pete,76.38,101.41,W,1,4,101.04,104.59,273.l.777818.t.7,Goons,Jake,114.56,88.40,L,0,5,81.10,76.48,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,100.96,83.30,L,0,6,122.04,107.65,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,66.74,109.33,W,1,7,61.30,69.07,273.l.777818.t.12,273.l.777818.t.12,273.l.777818.t.12,90.16,83.98,L,0,8,89.14,80.00,273.l.777818.t.11,La Flama Blanca,Ryan,104.48,93.96,L,1,9,121.58,95.66,273.l.777818.t.1,Taco,James,109.04,78.75,W,1,10,107.70,89.17,273.l.777818.t.2,273.l.777818.t.2,273.l.777818.t.2,68.08,95.08,W,1,

#MOM FFBL Playoff Bracket

In [2]:
endofseason_rankings_query = """SELECT game_id, reg_season_rank, manager, team_name, team_key FROM prod.regseasonrankings where game_id = 273"""
eos_rankings = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres(
    endofseason_rankings_query
)


league_settings_query = """SELECT playoff_start_week, num_playoff_teams, game_id from dev.leaguesettings where game_id = 273"""
league_settings = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    league_settings_query
)


team_points_weekly_query = (
    """SELECT * from dev.weeklyteampoints where game_id = 273 and week >= 12"""
)
team_points_weekly = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    team_points_weekly_query
)


teams_query = """SELECT team_key, name, nickname, game_id, league_id, team_id FROM dev.leagueteams where game_id = 273"""
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)


# team_points_weekly = team_points_weekly.merge(teams, how='outer', left_on=['game_id', 'league_id', 'team_id'], right_on=['game_id', 'league_id', 'team_id'])
# team_key = team_points_weekly['game_id'].astype(str)+"."+team_points_weekly['league_id'].astype(str)+".t."+team_points_weekly['team_id'].astype(str)
# team_points_weekly['team_key'].fillna(team_key, inplace=True)

Successfully pulled: SELECT game_id, reg_season_rank, manager, team_name, team_key FROM prod.regseasonrankings where game_id = 273
Successfully pulled: SELECT playoff_start_week, num_playoff_teams, game_id from dev.leaguesettings where game_id = 273
Successfully pulled: SELECT * from dev.weeklyteampoints where game_id = 273 and week >= 12
Successfully pulled: SELECT team_key, name, nickname, game_id, league_id, team_id FROM dev.leagueteams where game_id = 273


In [3]:
team_points_weekly

,final_points,week,projected_points,team_id,game_id,league_id
0,112.84,12,89.86,1,273,777818
1,74.64,12,96.24,11,273,777818
2,67.00,12,79.85,3,273,777818
3,104.28,12,87.15,4,273,777818
4,88.76,12,96.08,5,273,777818
5,112.14,12,103.52,6,273,777818
6,79.82,12,98.40,7,273,777818
7,86.24,13,82.95,1,273,777818
8,127.92,13,103.66,11,273,777818
9,75.98,13,88.47,3,273,777818


In [74]:
def playoff_winner(team1, team2, round, league_settings, points_df):

    playoff_week = league_settings["playoff_start_week"].values[0] + (round - 1)
    team1_score = points_df["final_points"][
        (points_df["team_key"] == team1) & (points_df["week"] == playoff_week)
    ].values[0]
    team2_score = points_df["final_points"][
        (points_df["team_key"] == team2) & (points_df["week"] == playoff_week)
    ].values[0]

    if team1_score > team2_score:
        winner = team1
        loser = team2
    elif team1_score < team2_score:
        winner = team2
        loser = team1
    else:
        winner = np.nan
        loser = np.nan

    return winner, loser


def games_round(games, round, league_settings, points_df):
    winners = []
    losers = []
    for team1, team2 in games:
        winning_team, losing_team = playoff_winner(
            team1, team2, round, league_settings, points_df
        )
        winners.append(winning_team)
        losers.append(losing_team)

    return winners, losers


def plan_games(teams):
    return zip(teams[::2], teams[1::2])


def first_place(eos_rankings, league_settings, points_df):

    if len(list(eos_rankings["game_id"].unique())) > 1:
        print(
            "Error, please only include 1 season into the rankings for bracket build."
        )

    else:
        game_id = eos_rankings["game_id"].values[0]
        rankings = eos_rankings.sort_values("reg_season_rank")
        first_place_teams = list(
            rankings["team_key"][
                (
                    rankings["reg_season_rank"]
                    <= league_settings["num_playoff_teams"][
                        league_settings["game_id"] == game_id
                    ].values[0]
                )
            ]
        )
        second_place_teams = []
        third_place_teams = []
        fourth_place_teams = []
        round = 0
        while len(first_place_teams) > 1:
            round += 1
            print(f"Round {round}: playoffs: {first_place_teams}")
            if round > 1:
                print(f"Round {round}: playoff consolation: {second_place_teams}")
            playoff_games = plan_games(first_place_teams)
            consolation_games = plan_games(second_place_teams)
            first_place_teams, second_place_teams = games_round(
                playoff_games, round, league_settings, points_df
            )
            third_place_teams, fourth_place_teams = games_round(
                consolation_games, round, league_settings, points_df
            )

        champion = first_place_teams[0]  # only one left
        second_place = second_place_teams[0]
        third_place = third_place_teams[0]
        fourth_place = fourth_place_teams[0]
        print(f"Champion is {champion}")
        print(f"Second Place is {second_place}")
        print(f"Third Place is {third_place}")
        print(f"Fourth Place is {fourth_place}")


first_place(
    eos_rankings=eos_rankings,
    league_settings=league_settings,
    points_df=team_points_weekly,
)

Round 1: playoffs: ['273.l.777818.t.7', '273.l.777818.t.10', '273.l.777818.t.2', '273.l.777818.t.6', '273.l.777818.t.9', '273.l.777818.t.4', '273.l.777818.t.12', '273.l.777818.t.1']


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams"""
players_query = """SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist"""
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
draft_analysis = draft.merge(
    teams,
    how="left",
    left_on=["team_key", "game_id", "league_id"],
    right_on=["team_key", "game_id", "league_id"],
    suffixes=("_draft", "_teams"),
)
draft_analysis = draft_analysis.merge(
    players,
    how="left",
    left_on=["player_key", "game_id", "league_id"],
    right_on=["player_key", "game_id", "league_id"],
    suffixes=("", "_players"),
)
draft_analysis = draft_analysis[~draft_analysis["name"].isna()]
draft_analysis["clinched_playoffs"].fillna(0, inplace=True)
draft_analysis["team_standings.playoff_seed"].fillna(0, inplace=True)
encoded_grades = {
    "A+": 4,
    "A": 4,
    "A-": 3.7,
    "B+": 3.3,
    "B": 3.0,
    "B-": 2.7,
    "C+": 2.3,
    "C": 2.0,
    "C-": 1.7,
    "D+": 1.3,
    "D": 1.0,
    "na": np.nan,
}
draft_analysis["draft_gpa"] = draft_analysis["draft_grade"].replace(encoded_grades)
draft_analysis["avg_draft_gpa_rank"] = draft_analysis.groupby(["team_standings.rank"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_nickname"] = draft_analysis.groupby(["nickname"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_clinched_playoffs"] = draft_analysis.groupby(
    ["clinched_playoffs"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_playoffseed"] = draft_analysis.groupby(
    ["team_standings.playoff_seed"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_draft_pos"] = draft_analysis.groupby(["pick"])[
    "draft_gpa"
].transform("mean")

MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft_query = """SELECT * FROM dev.draftresults"""
players_query = """SELECT * FROM dev.playerlist"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_a_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("", "_team_a"),
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_b_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("_team_a", "_team_b"),
)

matchups[
    (matchups["is_playoffs"] == 1) & (matchups["is_consolation"] == 0)
].sort_values(
    ["week_start", "is_consolation", "winner_team_key"], ascending=[False, True, True]
)

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""